In [1]:
import datajoint as dj
import numpy as np
from tqdm import tqdm

In [43]:
dj.config["display.limit"] = 31

ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
ndap100 = dj.create_virtual_module("ndap100","microns_ndap100")
schema = dj.schema("microns_ta3p100")

# GENERATE NEW CONTACT TEST TABLE FOR TESTING

In [3]:
key = dict(segmentation=2)
targets = (ta3p100.Segment & ndap100.Trace & key) - ta3p100.SegmentExclude
axons = ta3p100.NeuriteRevised & 'neurite_type="axon"' #REPLACE: neurites with neuriteRevised


In [4]:
#ta3p100.FinalSharedContactTest.drop()

In [5]:
@schema
class FinalSharedContactTest(dj.Computed):
    definition = """    
    -> ta3p100.Segment
    segment_b           : bigint   # segment id unique within each Segmentation
    ---
    n_cont_a     : int # number of synapses for cell A
    n_cont_b     : int # number of synapses for cell B
    n_cont_union : int # number of unique synapses for both
    n_cont_shared: int # number of shared synapses
    n_seg_a     : int # number of segments for cell A
    n_seg_b     : int # number of segments for cell B
    n_seg_union : int # number of unique segments for both
    n_seg_shared: int # number of shared segments
    """

    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
                #gets the current target (because restricts by key)
        targets = (ta3p100.Segment & ndap100.Trace & key) - ta3p100.SegmentExclude
        #all the possible neurites that could target onto the segment

        """  Things to change
        axons = dj.U("presyn") & ta3p100.Contacts
        """
        axons = ta3p100.NeuriteRevised & 'neurite_type="axon"' #REPLACE: neurites with neuriteRevised
        

        contact = (ta3p100.FinalContactTest & axons.proj(presyn='segment_id') & key) #- ta3p100.SynapseExcludeFinal
        info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
        
        A = (contact & targets.proj(postsyn='segment_id')).proj(
            'presyn', cont1='contact_id', segment_id='postsyn')
        B = (contact * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
            'presyn', cont2='contact_id', segment_b='postsyn')
        shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_cont_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')
        a = dj.U('segment_id').aggr(A, n_cont_a='count(*)', n_seg_a='count(DISTINCT presyn)')
        b = dj.U('segment_b').aggr(B, n_cont_b='count(*)', n_seg_b='count(DISTINCT presyn)')
        stats = (info * a * b * shared).proj('n_cont_a', 'n_cont_b', 'n_cont_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_cont_union='n_cont_a + n_cont_b - n_cont_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared')
        
        self.insert([dict(key, **k) for k in tqdm(stats.fetch(as_dict=True))],
                    ignore_extra_fields=True)
        
        non_shared = ((info - stats) * a * b).proj('n_cont_a', 'n_cont_b',
                                                    'n_seg_a', 'n_seg_b',
                                                    n_cont_union='n_cont_a + n_cont_b',
                                                    n_seg_union='n_seg_a + n_seg_b')
        self.insert([dict(key, **k, n_cont_shared=0, n_seg_shared=0) for k in tqdm(non_shared.fetch(as_dict=True))],ignore_extra_fields=True)
      

In [7]:
FinalSharedContactTest()

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation,n_cont_a number of synapses for cell A,n_cont_b number of synapses for cell B,n_cont_union number of unique synapses for both,n_cont_shared number of shared synapses,n_seg_a number of segments for cell A,n_seg_b number of segments for cell B,n_seg_union number of unique segments for both,n_seg_shared number of shared segments
2,648518346341352006,648518346341352223,39,31,-129,199,7,7,7,7
2,648518346341352006,648518346341353019,39,55,-246,340,7,7,7,7
2,648518346341352006,648518346341353186,39,40,-135,214,7,7,7,7
2,648518346341352006,648518346341353574,39,39,-141,219,7,7,7,7
2,648518346341352223,648518346341353019,31,55,-173,259,7,7,7,7
2,648518346341352223,648518346341353186,31,40,-98,169,7,7,7,7
2,648518346341352223,648518346341353574,31,39,-105,175,7,7,7,7


In [6]:
import time
start_time = time.time()
FinalSharedContactTest.populate()
print(f"Total seconds: {time.time()-start_time}")

100%|██████████| 10/10 [00:00<00:00, 15797.76it/s]
0it [00:00, ?it/s]

Total seconds: 8.567388772964478


In [8]:
targets = (ta3p100.Segment & ndap100.Trace & key) - ta3p100.SegmentExclude
#all the possible neurites that could target onto the segment

"""  Things to change
axons = dj.U("presyn") & ta3p100.Contacts
"""
axons = ta3p100.NeuriteRevised & 'neurite_type="axon"' #REPLACE: neurites with neuriteRevised


contact = (ta3p100.FinalContactTest & axons.proj(presyn='segment_id') & key) #- ta3p100.SynapseExcludeFinal
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
info

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation
2,648518346341352006,648518346341352223
2,648518346341352006,648518346341353019
2,648518346341352006,648518346341353186
2,648518346341352006,648518346341353574
2,648518346341352006,648518346341353607
2,648518346341352006,648518346341353788
2,648518346341352006,648518346341353883


In [9]:
A = (contact & targets.proj(postsyn='segment_id')).proj(
    'presyn', cont1='contact_id', segment_id='postsyn')
B = (contact * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
    'presyn', cont2='contact_id', segment_b='postsyn')
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                              n_cont_shared='count(*)',
                                              n_seg_shared='count(DISTINCT presyn)')

shared

segment_id,segment_b,n_cont_shared calculated attribute,n_seg_shared calculated attribute
648518346341352006,648518346341352223,199,7
648518346341352006,648518346341353019,340,7
648518346341352006,648518346341353186,214,7
648518346341352006,648518346341353574,219,7
648518346341352223,648518346341353019,259,7
648518346341352223,648518346341353186,169,7
648518346341352223,648518346341353574,175,7


# FinalContactTestMini

In [14]:
FinalSharedContactTestMini.drop()

`microns_ta3p100`.`__final_shared_contact_test_mini` (10 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [15]:
@schema
class FinalSharedContactTestMini(dj.Computed):
    definition = """    
    -> ta3p100.Segment
    segment_b           : bigint   # segment id unique within each Segmentation
    ---
    n_cont_a     : int # number of synapses for cell A
    n_cont_b     : int # number of synapses for cell B
    n_cont_union : int # number of unique synapses for both
    n_cont_shared: int # number of shared synapses
    n_seg_a     : int # number of segments for cell A
    n_seg_b     : int # number of segments for cell B
    n_seg_union : int # number of unique segments for both
    n_seg_shared: int # number of shared segments
    """

    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
                #gets the current target (because restricts by key)
        targets = (ta3p100.Segment & ndap100.Trace & key) - ta3p100.SegmentExclude
        #all the possible neurites that could target onto the segment

        """  Things to change
        axons = dj.U("presyn") & ta3p100.Contacts
        """
        axons = ta3p100.NeuriteRevised & 'neurite_type="axon"' #REPLACE: neurites with neuriteRevised
        

        contact = (ta3p100.FinalContactTestMini & axons.proj(presyn='segment_id') & key) #- ta3p100.SynapseExcludeFinal
        info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
        
        A = (contact & targets.proj(postsyn='segment_id')).proj(
            'presyn', cont1='contact_id', segment_id='postsyn')
        B = (contact * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
            'presyn', cont2='contact_id', segment_b='postsyn')
        shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_cont_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')
        a = dj.U('segment_id').aggr(A, n_cont_a='count(*)', n_seg_a='count(DISTINCT presyn)')
        b = dj.U('segment_b').aggr(B, n_cont_b='count(*)', n_seg_b='count(DISTINCT presyn)')
        stats = (info * a * b * shared).proj('n_cont_a', 'n_cont_b', 'n_cont_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_cont_union='n_cont_a + n_cont_b - n_cont_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared')
        
        self.insert([dict(key, **k) for k in tqdm(stats.fetch(as_dict=True))],
                    ignore_extra_fields=True)
        
        non_shared = ((info - stats) * a * b).proj('n_cont_a', 'n_cont_b',
                                                    'n_seg_a', 'n_seg_b',
                                                    n_cont_union='n_cont_a + n_cont_b',
                                                    n_seg_union='n_seg_a + n_seg_b')
        self.insert([dict(key, **k, n_cont_shared=0, n_seg_shared=0) for k in tqdm(non_shared.fetch(as_dict=True))],ignore_extra_fields=True)
      

In [40]:
#648518346341353521, 648518346341353986
#(ta3p100.SegmentExclude() & "criteria_id > 6" & "criteria_id < 10").delete()
(ta3p100.ExclusionCriteria() & "criteria_id>6" & "criteria_id<10").delete()

About to delete:
`microns_ta3p100`.`#exclusion_criteria`: 3 items


Proceed? [yes, No]:  yes


Committed.


In [18]:
import time
start_time = time.time()
FinalSharedContactTestMini.populate()
print(f"Total seconds: {time.time()-start_time}")

100%|██████████| 3/3 [00:00<00:00, 6156.02it/s]
0it [00:00, ?it/s]

Total seconds: 5.413415193557739


In [19]:
FinalSharedContactTestMini()

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation,n_cont_a number of synapses for cell A,n_cont_b number of synapses for cell B,n_cont_union number of unique synapses for both,n_cont_shared number of shared synapses,n_seg_a number of segments for cell A,n_seg_b number of segments for cell B,n_seg_union number of unique segments for both,n_seg_shared number of shared segments
2,648518346341352006,648518346341352223,6,12,-3,21,3,4,4,3
2,648518346341352006,648518346341353019,6,12,2,16,3,4,4,3
2,648518346341352223,648518346341353019,12,12,-4,28,4,4,4,4


In [22]:
postsyns = [
648518346341352006,
648518346341352223,
648518346341353019,
]

presyns = [648518346341351467,
648518346341351503,
648518346341351508,
648518346341351509,
]

#check all of these in the tables

In [26]:
key #key is just the segmentation

{'segmentation': 2}

In [41]:
targets = (ta3p100.Segment & ndap100.Trace & key) - ta3p100.SegmentExclude
#all the possible neurites that could target onto the segment

"""  Things to change
axons = dj.U("presyn") & ta3p100.Contacts
"""
axons = ta3p100.NeuriteRevised & 'neurite_type="axon"' #REPLACE: neurites with neuriteRevised


contact = (ta3p100.FinalContactTestMini & axons.proj(presyn='segment_id') & key) #- ta3p100.SynapseExcludeFinal
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
info

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation
2,648518346341352006,648518346341352223
2,648518346341352006,648518346341353019
2,648518346341352006,648518346341353186
2,648518346341352006,648518346341353574
2,648518346341352006,648518346341353607
2,648518346341352006,648518346341353788
2,648518346341352006,648518346341353883


In [24]:
(targets & [dict(segment_id=x) for x in postsyns]) #so checks out that they exist in the tables

segmentation segmentation id,segment_id segment id unique within each Segmentation
2,648518346341352006
2,648518346341352223
2,648518346341353019


In [25]:
(axons & [dict(segment_id=x) for x in presyns]) #so checks out that they exist in the tables

segmentation segmentation id,segment_id segment id unique within each Segmentation,neurite_type neurite type
2,648518346341351467,axon
2,648518346341351503,axon
2,648518346341351508,axon
2,648518346341351509,axon


In [58]:
B

segmentation,cont2,presyn,segment_b
2,1,648518346341351467,648518346341352006
2,15,648518346341351508,648518346341352006
2,17,648518346341351508,648518346341352006
2,21,648518346341351509,648518346341352006
2,23,648518346341351508,648518346341352006
2,27,648518346341351509,648518346341352006
2,2,648518346341351503,648518346341352223
2,3,648518346341351509,648518346341352223
2,6,648518346341351509,648518346341352223
2,7,648518346341351508,648518346341352223


In [60]:
#WHAT IS THIS DOING: taking every contact and finding all the different combinations of contact 
### takes every presyn with segmen_id pair and creates new row with all other segments that have same presyn 
### and adds that as a shared row that gets summed into shared contacts

#Can easily result in there being higher where n_cont - shared

A * B & 'segment_id < segment_b'

segmentation,cont1,cont2,presyn,segment_id,segment_b
2,1,29,648518346341351467,648518346341352006,648518346341352223
2,1,19,648518346341351467,648518346341352006,648518346341353019
2,1,24,648518346341351467,648518346341352006,648518346341353019
2,1,5,648518346341351467,648518346341352006,648518346341353019
2,1,9,648518346341351467,648518346341352006,648518346341353019
2,1,26,648518346341351467,648518346341352006,648518346341353019
2,1,18,648518346341351467,648518346341352006,648518346341353019
2,15,28,648518346341351508,648518346341352006,648518346341352223
2,15,7,648518346341351508,648518346341352006,648518346341352223
2,15,12,648518346341351508,648518346341352006,648518346341352223


In [42]:
A = (contact & targets.proj(postsyn='segment_id')).proj(
    'presyn', cont1='contact_id', segment_id='postsyn')
B = (contact * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
    'presyn', cont2='contact_id', segment_b='postsyn')
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                              n_cont_shared='count(*)',
                                              n_seg_shared='count(DISTINCT presyn)')

shared

#CONCLUSION: MAKES SENSE BECAUSE SHOULD BE 3 NCR 3 and shows thtat they all share the same segments


segment_id,segment_b,n_cont_shared calculated attribute,n_seg_shared calculated attribute
648518346341352006,648518346341352223,21,3
648518346341352006,648518346341353019,16,3
648518346341352223,648518346341353019,28,4


In [48]:
dj.U("presyn")& (ta3p100.FinalContactTestMini().proj("presyn","postsyn") & "postsyn=648518346341352006")

presyn
648518346341351467
648518346341351508
648518346341351509


In [49]:
dj.U("presyn")& (ta3p100.FinalContactTestMini().proj("presyn","postsyn") & "postsyn=648518346341352223")

presyn
648518346341351503
648518346341351509
648518346341351508
648518346341351467


In [50]:
dj.U("presyn")& (ta3p100.FinalContactTestMini().proj("presyn","postsyn") & "postsyn=648518346341353019")

presyn
648518346341351509
648518346341351503
648518346341351467
648518346341351508


In [72]:
#ta3p100.NeuriteRevised()

In [70]:
226801/7

32400.14285714286

In [67]:
info #all the possible combinations without repeats for segments with postsyn specified and Neurite
stats = (info) * a #all po
#dj.U("segment_id") & stats
info

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation
2,648518346341352006,648518346341352223
2,648518346341352006,648518346341353019
2,648518346341352006,648518346341353186
2,648518346341352006,648518346341353574
2,648518346341352006,648518346341353607
2,648518346341352006,648518346341353788
2,648518346341352006,648518346341353883
2,648518346341352006,648518346341354048
2,648518346341352006,648518346341354962
2,648518346341352006,648518346341355539


In [51]:
a = dj.U('segment_id').aggr(A, n_cont_a='count(*)', n_seg_a='count(DISTINCT presyn)')
b = dj.U('segment_b').aggr(B, n_cont_b='count(*)', n_seg_b='count(DISTINCT presyn)')
stats = (info * a * b * shared).proj('n_cont_a', 'n_cont_b', 'n_cont_shared',
                                     'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                     n_cont_union='n_cont_a + n_cont_b - n_cont_shared',
                                     n_seg_union='n_seg_a + n_seg_b - n_seg_shared')

stats

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation,n_cont_a calculated attribute,n_seg_a calculated attribute,n_cont_b calculated attribute,n_seg_b calculated attribute,n_cont_shared calculated attribute,n_seg_shared calculated attribute,n_cont_union calculated attribute,n_seg_union calculated attribute
2,648518346341352006,648518346341352223,6,3,12,4,21,3,-3,4
2,648518346341352006,648518346341353019,6,3,12,4,16,3,2,4
2,648518346341352223,648518346341353019,12,4,12,4,28,4,-4,4


In [55]:
21 + 16 + 28

65

In [53]:
non_shared = ((info - stats) * a * b).proj('n_cont_a', 'n_cont_b',
                                                'n_seg_a', 'n_seg_b',
                                                n_cont_union='n_cont_a + n_cont_b',
                                                n_seg_union='n_seg_a + n_seg_b')
non_shared

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation,n_cont_a calculated attribute,n_seg_a calculated attribute,n_cont_b calculated attribute,n_seg_b calculated attribute,n_cont_union calculated attribute,n_seg_union calculated attribute


In [ ]:
Conclusion: Everything seems to be working perfectly fine